### Install Dependencies

In [4]:
# !pip install langdetect geoip2 awswrangler
# !pip install python-mimeparse
!pwd

  Using cached geoip2-5.1.0-py3-none-any.whl.metadata (19 kB)
  Using cached maxminddb-2.7.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.0 kB)
Using cached geoip2-5.1.0-py3-none-any.whl (27 kB)
Using cached maxminddb-2.7.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (88 kB)
/home/sagemaker-user/snitchmail-eda


In [11]:
import os
import json
import gzip
import shutil
import pandas as pd
from email.header import decode_header, make_header
from langdetect import detect
import geoip2.database
from collections import defaultdict
from multiprocessing import Pool
import email
from email import policy
from email.parser import BytesParser
import io
import time

import re
import codecs

from bs4 import XMLParsedAsHTMLWarning
import warnings
from bs4 import BeautifulSoup

import awswrangler as wr
wr.engine.set("python") 
import logging
logging.getLogger('nltk').setLevel(logging.ERROR)

warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

### Connect to Sendgrid Role

In [9]:
import boto3
from pprint import pprint

sts = boto3.Session().client("sts", region_name="us-east-1")
response = sts.assume_role(
    RoleArn="arn:aws:iam::375084544312:role/mimesample_delegate",
    RoleSessionName="mimesamples-access"
)

ACCESS_KEY = response["Credentials"]["AccessKeyId"]
SECRET_KEY = response["Credentials"]["SecretAccessKey"]
SESSION_TOKEN = response["Credentials"]["SessionToken"]

session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN
)
s3 = session.client('s3')

# Your bucket name
bucket_name = 'mime-samples-production'

In [10]:
COMPRESSED_DATA_PATH = './raw_emails/'
JSON_PATH = './raw_emails_json/'
#create folder if not already
os.makedirs(COMPRESSED_DATA_PATH, exist_ok=True)
os.makedirs(JSON_PATH, exist_ok=True)
# for f in os.listdir(EMAIL_PATH):
#     os.remove(os.path.join(EMAIL_PATH, f))
files_in_compressed_data = os.listdir(COMPRESSED_DATA_PATH)
files_in_json_data = os.listdir(JSON_PATH)

In [11]:
len(files_in_compressed_data), len(files_in_json_data)

(4, 3)

In [36]:
# paginator = s3.get_paginator('list_objects_v2')
# pages = paginator.paginate(Bucket=bucket_name, Prefix='users/50733733/')
file_list = wr.s3.list_objects("s3://mime-samples-production/users/50733733/", boto3_session=session)
for file in file_list:
    print(file)
# pages = paginator.paginate(Bucket=bucket_name, Prefix='json/day=20250412/')

s3://mime-samples-production/users/50733733/4b3544d1-837b-4995-87a6-5d9ad434f216.json
s3://mime-samples-production/users/50733733/c57823f7-4fe1-43a0-a231-ae0fb1a7e433.json
s3://mime-samples-production/users/50733733/cda86c6d-f6c3-474f-9644-29175d574016.json
s3://mime-samples-production/users/50733733/dafed507-8e22-4ad5-ba83-63e753d4778a.json


In [47]:
# for page in pages:
#     for obj in page['Contents']:
for key in file_list:
        user = key.split('/')[4]
        user_base_path = os.path.join(JSON_PATH, 'users', user)
        if not os.path.exists(user_base_path):
            os.mkdir(user_base_path)
        local_filename = os.path.join(user_base_path, key.split('/')[5])  # extract just the filename
        print(f"Downloading: {key} to {local_filename}")
        
        # Download file from S3 to local
        wr.s3.download(key, local_filename, boto3_session=session)
        print(f"Downloaded {local_filename} successfully.")

Downloading: s3://mime-samples-production/users/50733733/4b3544d1-837b-4995-87a6-5d9ad434f216.json to ./raw_emails_json/users/50733733/4b3544d1-837b-4995-87a6-5d9ad434f216.json
Downloaded ./raw_emails_json/users/50733733/4b3544d1-837b-4995-87a6-5d9ad434f216.json successfully.
Downloading: s3://mime-samples-production/users/50733733/c57823f7-4fe1-43a0-a231-ae0fb1a7e433.json to ./raw_emails_json/users/50733733/c57823f7-4fe1-43a0-a231-ae0fb1a7e433.json
Downloaded ./raw_emails_json/users/50733733/c57823f7-4fe1-43a0-a231-ae0fb1a7e433.json successfully.
Downloading: s3://mime-samples-production/users/50733733/cda86c6d-f6c3-474f-9644-29175d574016.json to ./raw_emails_json/users/50733733/cda86c6d-f6c3-474f-9644-29175d574016.json
Downloaded ./raw_emails_json/users/50733733/cda86c6d-f6c3-474f-9644-29175d574016.json successfully.
Downloading: s3://mime-samples-production/users/50733733/dafed507-8e22-4ad5-ba83-63e753d4778a.json to ./raw_emails_json/users/50733733/dafed507-8e22-4ad5-ba83-63e753d477

In [ ]:
date_base_path = 'raw_emails/dates/20250222/'
os.makedirs(date_base_path, exist_ok=True)
location = f"s3://mime-samples-production/json/day=20250402"
keys = wr.s3.list_objects(location, boto3_session=session)
files_in_local = os.listdir(date_base_path)
for key in keys:
    local_filename = os.path.join(date_base_path, key.split('/')[-1])  # extract just the filename
    if not key.split('/')[-1] in files_in_local:
        print(f"Downloading: {key} to {local_filename}")
        wr.s3.download(key, local_filename, boto3_session=session)
        print(f"Downloaded {local_filename} successfully.")

Downloading: s3://mime-samples-production/json/day=20250402/email-content-raw-production-1-2025-04-02-00-04-57-3557f507-6d37-48ec-91dc-f86274bfdca5.gz to raw_emails/dates/20250222/email-content-raw-production-1-2025-04-02-00-04-57-3557f507-6d37-48ec-91dc-f86274bfdca5.gz
Downloaded raw_emails/dates/20250222/email-content-raw-production-1-2025-04-02-00-04-57-3557f507-6d37-48ec-91dc-f86274bfdca5.gz successfully.
Downloading: s3://mime-samples-production/json/day=20250402/email-content-raw-production-1-2025-04-02-00-04-59-ad502137-a185-4b45-aed2-c142b275f145.gz to raw_emails/dates/20250222/email-content-raw-production-1-2025-04-02-00-04-59-ad502137-a185-4b45-aed2-c142b275f145.gz
Downloaded raw_emails/dates/20250222/email-content-raw-production-1-2025-04-02-00-04-59-ad502137-a185-4b45-aed2-c142b275f145.gz successfully.
Downloading: s3://mime-samples-production/json/day=20250402/email-content-raw-production-1-2025-04-02-00-05-01-7cf3efaa-550d-4162-9e88-a5d2f74de3d1.gz to raw_emails/dates/202

## S3 File Structure
1 page = 1000 files
1 file = .gz file --> unzip --> json 
1 json file ~= 2000 emails

json/day=20250412/<filename>.gz --> 1 JSON file --> multiple emails (each line = 1 email) ~ 2000

users/<userid>/<filename>.json --> single email in 1 line = 1 email

Suspended User  Suspension Date
<user_id>       <date>

emails <date> - 3 days to <date> = Phish

- number of users
- emails per date

## User-wise CSV generation

# Snowflake Connection to get Suspended Accounts

In [6]:
import snowflake.connector

conn = snowflake.connector.connect(
    user='monica.tare@sendgrid.com',
    account='sendgrid.us-east-1',
    authenticator= 'externalbrowser',
    warehouse='TRUST_INSIGHTS_STAGING_WH',
    database='TRUST_INSIGHTS_PROD',
    role='MDS_UNMASKED_ACCESS_ROLLUP',  # TRUST_INSIGHTS_DEVELOPER
    schema="trust_insights_prod_schema"
)

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://sendgrid.us-east-1.snowflakecomputing.com/console/login?login_name=monica.tare%40sendgrid.com&browser_mode_redirect_port=58985&proof_key=22mdakOp6v3BVXcvalTAlaMhu1R6ACOWiNkZRy4ujOo%3D to authenticate...
We were unable to open a browser window for you, please open the url above manually then paste the URL you are redirected to into the terminal.


Enter the URL the SSO URL redirected you to:  http://localhost:58985/?token=7VlZc9rKEn7nV7jIo8rWLhAV-14tLAIkEFpY3rSMFrSikZDg11-BjWM7OUnOSereqlspqmymZ7qnu79W8zV8_leTxHdHUMAwSx-7-APWvQOpk7lh6j92DX103-_-6-kztJKYyAcrAPMsheBOBLAMU6u8KgVlmcMBikKQun4Rug8VvAcWLO_xB5hmtRdbEXCyJK9aFf-hfYd6wEXjzA_T7p0kPnZDl2J6FE0xGEP1WIZm2xfNYjhL0RjNtmfS28V69tjNo9DFfTo6hIHDly5h1z3RqnKwi5ZTL3QTy5dsm3fy07C_FwPDafUhrICUwtJKy8cugRH0PUbdE6yOsQOCHZD0Q58hd90785YG4pKGNjEpHDxH_titinSQWTCEg9RKAByUzkDj5PmgPTrIi6zMnCy-6TTwOSltTuq6fqjJh6zwUQLDcHQjzzUnAInVfUnq4OpccTfKisQqv3_PRRK699716ACkZVie3vn5fXULQlBcIOs-vfEui0rrCgpoIrzoVawFe6m6H21N3O1_Rt86-fTZhQMt9FvgqwK8XOz-VbAYirFoe8aFof-p-6oLXCn1sutSsNIsDR0rDs_XUpJBGWTuHRf7WRGWQfKXWcSxi-F70Dj3Dk6ln7roe9d-2hBG3Ty8T7ICfCqgdQ8Di6CZF5Mr4IGifSDAnbGSHrufflCqVx29sFJ4AQm-X_7Qn3cZA-kRxFkO3Ht4C-vFp583-I1MPX0GzkBKnbiC4REolyLJLQfAu2UBvLCZh7Atwgbeygo4fwsE9K17H5bP2RBDv-0dfxOgFoBP72B5tmJacQWeCL2f5YxtjJA-1ePn-9jvZWhVl7ovGvM1Ao1CO9vccEnz28erR2-Vr4JXkJ-XH6r0taqeNUaZGp_jYwWXKZ_GUGCGggVl1qiyWe-kE83k5KZ0sQ88

In [85]:
import pandas as pd
def query_snowflake(query):
    df = pd.read_sql(
        query,
        conn
    )
    df.columns = [col.lower() for col in df.columns]
    return df

In [17]:
query = """
select * from edo_dw_prod.acq_maildb.acq_hist_maildb_admin_note
where agent_notes like '%DIRMBS:RElSTUJT%'
    and created_at >= dateadd(month, -2, current_timestamp)
"""
df = query_snowflake(query)

/tmp/ipykernel_8107/1449233593.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [96]:
## active_users
query = """
select user_id, count(*) email_count from "DP_MDS_RAW_EVENTS_DATABASE_PROD"."DP_MDS_RAW_EVENTS_SCHEMA"."GLADYS_LOG"
where created >= dateadd(month, -2, current_timestamp)
group by user_id
"""
active_users_df = query_snowflake(query)

/tmp/ipykernel_8107/1449233593.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [104]:
active_users_df[active_users_df['email_count'] > 1000].shape

(842929, 2)

In [18]:
df.columns # agent_notes, account column = user_id

Index(['etl_acq_row_num', 'etl_src_id', 'etl_src_abbrv', 'etl_src_table_name',
       'etl_src_start_date', 'etl_src_end_date', 'etl_uid_code', 'etl_md5',
       'etl_ins_run_id', 'etl_upd_run_id', 'etl_ins_date', 'etl_upd_date',
       'etl_del_date', 'id', 'user_id', 'agent_id', 'admin_notes_category_id',
       'package_id', 'package_name', 'parent_id', 'linked_note_id',
       'agent_notes', 'created_at', 'updated_at'],
      dtype='object')

In [20]:
users = df[['user_id', 'created_at']]

In [23]:
suspended_users = df.loc[
    df.groupby('user_id')['created_at'].idxmax(),
    ['user_id', 'created_at']
].reset_index(drop=True)

In [24]:
suspended_users

,user_id,created_at
0,3956193,2025-03-05 19:20:32
1,4828170,2025-03-13 08:18:54
2,7955388,2025-03-21 20:41:19
3,13503997,2025-02-24 13:25:19
4,23017485,2025-02-24 08:37:59
...,...,...
801,52181849,2025-04-22 15:24:32
802,52183024,2025-04-22 15:24:32
803,52184050,2025-04-22 16:34:13
804,52184741,2025-04-22 16:34:13


In [25]:
suspended_users.to_csv('csv_files/suspended_users_20250424.csv', index=False)

In [26]:
unique_users = list(suspended_users['user_id'].unique())

In [63]:
len(unique_users)

806

In [50]:
def process_json_lines(json_lines, s3_loc):
    local_data = {
        'user_id': [], 'msg_id': [], 'sg_event_id': [], 'subject': []
        , 'mime': [], 'email_from': [], 'email_to': [], 'email_date': []
        , 'originating_ip': [], 'originating_ip_country': [], 'lang': []
        , 's3_path': []
    }
    
    p = PreProcessor()

    for j in json_lines:
        try:
            raw_mime = j["raw_mime"]

            # Subject
            subject = str(make_header(decode_header(j["event"]["payload"]["subject"]))) if j["event"]["payload"]["subject"] else None

            # GeoIP setup per process
            with geoip2.database.Reader('./GeoIP2-Country.mmdb') as reader:
                originating_ip = j["event"]["payload"]["originating_ip"]
                try:
                    country = reader.country(originating_ip).country.name
                except:
                    country = 'N/A'

            # Language detection
            lang = detect(subject) if subject else 'N/A'

            # Append data
            local_data['user_id'].append(j["event"]["payload"]["userid"])
            local_data['msg_id'].append(j["event"]["payload"]["msgid"])
            local_data['sg_event_id'].append(j["event"]["payload"]["sg_event_id"])
        
            local_data['email_from'].append(j["event"]["payload"]["from"])
            local_data['email_to'].append(j["event"]["payload"]["email"])
            local_data['email_date'].append(j["event"]["payload"]["date"])
            
            local_data['subject'].append(subject)
            # local_data['html_file'].append(body_filename)
            local_data['mime'].append(raw_mime)
            local_data['originating_ip'].append(j["event"]["payload"]["originating_ip"])
            local_data['originating_ip_country'].append(country)
            local_data['lang'].append(lang)
            local_data['s3_path'].append(s3_loc)

        except Exception as e:
            print(f"Error in {s3_loc}: {str(e)}")
    return local_data

In [60]:
user_email_count = defaultdict(int)
for user in unique_users:
    if user not in user_email_count:
        user_emails = defaultdict(list)
        location = f"s3://mime-samples-production/users/{user}"
        files = wr.s3.list_objects(location, boto3_session=session)
        for filename in files:
            print(f"Reading: {filename}")
            
            # Read file from S3 in memory
            key = '/'.join(filename.split('/')[3:])
            obj = s3.get_object(Bucket=bucket_name, Key=key)
            lines = obj['Body'].read().decode("utf-8").splitlines()
            json_objects = [json.loads(line) for line in lines]
            user_emails[key] = json_objects
            
        with Pool() as pool:
            results = pool.starmap(process_json_lines, [(user_emails[s3_loc], s3_loc) for s3_loc in user_emails])
            
        # Aggregate results
        if results:
            processed = {k: [] for k in results[0]}
            for result in results:
                for k, v in result.items():
                    processed[k].extend(v)       
            df = pd.DataFrame(processed)
            df = df.sort_values(['sg_event_id'])
    
            csv_output = 'csv_files/wo_label/user_wise/'
            os.makedirs(csv_output, exist_ok=True)
        
            output_filename = os.path.join(csv_output, f'data_woLabel_{user}.csv')
            df.to_csv(output_filename, index=False)
            user_email_count[user] = len(df)
            print(f"\n\nProcessed CSV with DF of size {len(df)} and saved to {output_filename}!\n")
        else:
            user_email_count[user] = 0
            print(f"No files found for {user}: {files}")

Reading: s3://mime-samples-production/users/3956193/05df04f3-b0c5-4249-bc8c-fede78cfd990.json
Reading: s3://mime-samples-production/users/3956193/05f9bcf0-7fe5-4cf0-9793-0deb09120de4.json
Reading: s3://mime-samples-production/users/3956193/06b648f5-4da8-4c48-b5ee-f09e0acbb32a.json
Reading: s3://mime-samples-production/users/3956193/0a010116-fa72-46ad-9e8b-86f61d917dfd.json
Reading: s3://mime-samples-production/users/3956193/0b3c405f-a6ea-42fc-9f30-6a2fe8c09048.json
Reading: s3://mime-samples-production/users/3956193/1311f2e2-cd28-4a77-bb59-2653f3dde436.json
Reading: s3://mime-samples-production/users/3956193/1561d429-37aa-4e92-85a0-8d761b706991.json
Reading: s3://mime-samples-production/users/3956193/161cdb2d-701c-466c-b278-900a0c471476.json
Reading: s3://mime-samples-production/users/3956193/191817d2-e513-4d5a-aaf7-75d3fcdb5f14.json
Reading: s3://mime-samples-production/users/3956193/1b3e8b76-4f84-4fa0-aa99-439942b52891.json
Reading: s3://mime-samples-production/users/3956193/1b920e66

In [61]:
len(user_email_count)

806

In [62]:
user_stats = pd.DataFrame()
user_stats['user'] = list(user_email_count.keys())
user_stats['email_count'] = list(user_email_count.values())

In [65]:
user_stats.to_csv('csv_files/user_stats_20250424.csv', index=False)

## Process Suspended Users
* Read raw mime
* Extract text and html
* Save HTML file to S3
* Prepare combined parque
* Save to S3

In [161]:
SUS_USER_PATH = 'csv_files/wo_label/user_wise/'
user_files = os.listdir(SUS_USER_PATH)

In [4]:
def extract_data_from_raw_mime(raw_mime, msg_id):
    raw_mime = raw_mime.encode('utf-8')
    msg = BytesParser(policy=policy.default).parsebytes(raw_mime)
    
    # from_ = msg['From']
    # to = msg['To']
    # subject = msg['Subject']
    # date = msg['Date']
    content_type = msg.get_content_type()
    text_body = ""
    html_body = ""
    
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = part.get("Content-Disposition", "")
            if content_type == "text/plain" and "attachment" not in content_disposition:
                text_body = part.get_payload(decode=True).decode(part.get_content_charset('utf-8'))
            elif content_type == "text/html":
                html_body = part.get_payload(decode=True).decode(part.get_content_charset('utf-8'))
    else:
        content_type = msg.get_content_type()
        if content_type == "text/plain":
            text_body = msg.get_payload(decode=True).decode(msg.get_content_charset('utf-8'))
        elif content_type == "text/html":
            html_body = msg.get_payload(decode=True).decode(msg.get_content_charset('utf-8'))

    # Optional: Save HTML to a file
    if html_body:
        # os.makedirs("html_files", exist_ok=True)
        # with open(f"html_output/{filename}.html", "w", encoding='utf-8') as f:
        #     f.write(html_body)
        # s3_uri = "s3://accsec-ai-prod-snitchmail/processed/phish/html_files/"  # ensure trailing slash
        s3_uri = "s3://accsec-ai-prod-snitchmail/processed/not_phish/html_files/"
        
        # Upload the file
        dest_path = s3_uri + f"{msg_id}.html"
        buffer = io.BytesIO(html_body.encode('utf-8'))
        wr.s3.upload(
            local_file=buffer,
            path=dest_path
        )
        print(f"Uploaded to {dest_path}")

    return {
        # "from_address": from_,
        # "to_address": to,
        # "subject": subject,
        # "Date": date,
        "Content-Type": content_type,
        "email_text": text_body
    }

In [164]:
processed_dfs = []
for i, filename in enumerate(user_files):
    if filename.endswith('.csv'):
        file_path = os.path.join(SUS_USER_PATH, filename)
        user_df = pd.read_csv(file_path)

        # Apply the function to each row, passing 'mime' and 'msg_id'
        processed_df = user_df.apply(lambda row: extract_data_from_raw_mime(row['mime'], row['msg_id']), axis=1)
        
        # Convert the result (assuming dict or Series) into DataFrame
        processed_df = processed_df.apply(pd.Series)
        
        # Concatenate the extracted columns with original DataFrame
        df = pd.concat([user_df, processed_df], axis=1)
        if len(df) > 0:
            processed_dfs.append(df)


Uploaded to s3://accsec-ai-prod-snitchmail/processed/phish/html_files/0OK0-wLUSHqT1AabKOESZQ.recvd-8575c86f9d-7252j-1-67D4ABFE-2B.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/phish/html_files/1ebF7IUZR1-6rVSsA3tfCQ.recvd-69cbc48b94-7j96v-1-67BA03D1-35.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/phish/html_files/7EXFhslEReuk9TB36V8-8g.recvd-578d769b87-hhkrc-1-67D32040-4.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/phish/html_files/7tZbMk1STLGD4PiHZUUJUQ.recvd-69cbc48b94-2tgq2-1-67BB28A6-5.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/phish/html_files/4ZulGs9DTv6eQTMj05NAfg.recvd-97d7ddb5-h84mg-1-67DDE06A-26.1.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/phish/html_files/5dSNt7SfSjegRmAXP-KtBw.recvd-97d7ddb5-hb2sp-1-67E014ED-6.1.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/phish/html_files/664BdogfSRWodMiDaUau1A.recvd-69cbc48b94-5zxfq-1-67C65E1B-2.3.html
Uploaded to s3://accsec-ai-prod-snitchmail/proces

In [166]:
len(processed_dfs)

329

In [167]:
# Ensure unique indices and columns before concatenation
processed_dfs = [df.reset_index(drop=True) for df in processed_dfs]
for i, df in enumerate(processed_dfs):
    if not df.columns.is_unique:
        # Optionally: df = df.loc[:,~df.columns.duplicated()]
        print(i)
        raise ValueError(f"Duplicate columns: {df.columns[df.columns.duplicated()]}")

phish_processed_df = pd.concat(processed_dfs, ignore_index=True)
# No need for further index resetting
print(phish_processed_df.shape)

(5599, 14)


In [189]:
# phish_processed_df[phish_processed_df['Content-Type'] == 'text/plain']
phish_processed_df.loc[259]

user_id                                                            32338879
msg_id                    -ZFf_SSnRNGCX_cOoNgxgQ.recvd-69cbc48b94-hnb44-...
sg_event_id               cHJvY2Vzc2VkLTMyMzM4ODc5LS1aRmZfU1NuUk5HQ1hfY0...
subject                                          Amdromeda "[your-subject]"
mime                      DKIM-Signature: v=1; a=rsa-sha256; c=relaxed/r...
email_from                                 "Amdromeda" <info@amdromeda.com>
email_to                                       enmanuel.laraperez@gmail.com
email_date                                                       2025-02-23
originating_ip                                               67.217.241.172
originating_ip_country                                        United States
lang                                                                     en
s3_path                   users/32338879/9c521c2e-ffa9-46da-8d61-b74a422...
Content-Type                                                     text/plain
email_text  

## Process Safe Users

In [199]:
def process_user(user_id):
    """Process a single user: fetch, parse, extract, upload HTML, return record dict."""
    s3_loc = f"s3://mime-samples-production/users/{user_id}/"
    # List all files for this user, pick the first
    files = wr.s3.list_objects(s3_loc, boto3_session=session)
    if not files:
        return None
    json_file = files[0]  # Pick the first file
    
    # Read JSON file (assume one record per file)
    bucket_name = 'mime-samples-production'
    obj = s3.get_object(Bucket=bucket_name, Key=json_file)
    lines = obj['Body'].read().decode("utf-8").splitlines()
    json_objects = [json.loads(line) for line in lines]
    if not json_objects:
        return None
    j = json_objects[0]
    
    # Extract fields
    local_data = {}
    try:
        raw_mime = j["raw_mime"]

        # Subject
        subject = str(make_header(decode_header(j["event"]["payload"]["subject"]))) if j["event"]["payload"]["subject"] else None

        # GeoIP setup per process
        with geoip2.database.Reader('./GeoIP2-Country.mmdb') as reader:
            originating_ip = j["event"]["payload"]["originating_ip"]
            try:
                country = reader.country(originating_ip).country.name
            except:
                country = 'N/A'

        # Language detection
        lang = detect(subject) if subject else 'N/A'

        # Append data
        local_data['user_id'] = j["event"]["payload"]["userid"]
        local_data['msg_id'] = j["event"]["payload"]["msgid"]
        local_data['sg_event_id'] = j["event"]["payload"]["sg_event_id"]
    
        local_data['email_from'] = j["event"]["payload"]["from"]
        local_data['email_to'] = j["event"]["payload"]["email"]
        local_data['email_date'] = j["event"]["payload"]["date"]
        
        local_data['subject'] = subject
        local_data['mime'] = raw_mime
        local_data['originating_ip'] = j["event"]["payload"]["originating_ip"]
        local_data['originating_ip_country'] = country
        local_data['lang'] = lang
        local_data['s3_path'] = json_file
        
        processed_data = extract_data_from_raw_mime(raw_mime, local_data['msg_id'])
        local_data['Content-Type'] = processed_data['Content-Type']
        local_data['email_text'] = processed_data['email_text']

    except Exception as e:
        print(f"Error in {json_file}: {str(e)}")
    # Prepare record for DataFrame
    return local_data

In [200]:
def process_batch(user_ids, batch_start, batch_end):
    """Process a batch of user_ids in parallel and write to S3 as Parquet."""
    with mp.Pool(processes=os.cpu_count()) as pool:
        results = pool.map(process_user, user_ids)
    # Filter out None results
    records = [r for r in results if r]
    if records:
        df = pd.DataFrame(records)
        batch_id = f"{batch_start}-{batch_end}"
        OUTPUT_S3_BUCKET = "s3://accsec-ai-prod-snitchmail/processed/not_phish/"
        parquet_path = f"{OUTPUT_S3_BUCKET}/non_phish_users_sample_{batch_id}.parquet"
        wr.s3.to_parquet(df, parquet_path)

In [ ]:
with open('pending_list.txt', 'r', encoding='utf-8') as f:
    user_ids = [line.strip() for line in f if line.strip()]
user_ids = user_ids[::-1]

# Process in batches of PARQUET_BATCH_SIZE
PARQUET_BATCH_SIZE = 5000
for i in range(0, len(user_ids), PARQUET_BATCH_SIZE):
    batch = user_ids[i: i + PARQUET_BATCH_SIZE]
    process_batch(batch, i, i + PARQUET_BATCH_SIZE - 1)

## Process Saved Not Phish User Data

In [13]:
# s3_parquet_path = "s3://accsec-ai-prod-snitchmail/processed/not_phish/non_phish_sample_125k.parquet"
s3_parquet_path = "s3://accsec-ai-prod-snitchmail/processed/not_phish/unique_user_email_data/partial_0_0.parquet"
unique_email_0_df = wr.s3.read_parquet(path=s3_parquet_path)
# Load the Parquet file as a DataFrame
# good_users_125k_df = wr.s3.read_parquet(path=s3_parquet_path)

In [15]:
# good_users_125k_df.head()
unique_email_0_df.shape

(500, 72)

In [204]:
# good_user_0_3866 = pd.DataFrame()

In [206]:
phish_processed_df.columns

Index(['user_id', 'msg_id', 'sg_event_id', 'subject', 'mime', 'email_from',
       'email_to', 'email_date', 'originating_ip', 'originating_ip_country',
       'lang', 's3_path', 'Content-Type', 'email_text'],
      dtype='object')

In [17]:
!ls

GeoIP2-Country.mmdb  __pycache__	data.ipynb	  preprocessor.py
LICENSE		     awswrangler.ipynb	pending_list.txt  raw_emails
README.md	     csv_files		phish_html	  raw_emails_json


In [5]:
import geoip2.database

def get_country(ip):
    if not ip:
        return 'N/A'
    try:
        with geoip2.database.Reader('./GeoIP2-Country.mmdb') as reader:
            return reader.country(ip).country.name
    except Exception:
        return 'N/A'

def detect_lang(text):
    try:
        return detect(text) if pd.notnull(text) and text else 'N/A'
    except Exception:
        return 'N/A'

In [6]:
def process_5k_good_users(args):
    start = time.time()
    file_group, batch_start, batch_end = args
    batch_id = f"{batch_start}-{batch_end}"
    parquet_path = f"s3://accsec-ai-prod-snitchmail/processed/not_phish/unique_user_email_data_processed/ \
users_sample_{batch_id}.parquet"
    
    # Read only the required files for this batch
    batch_df = wr.s3.read_parquet(path=file_group)
    
    good_users_5k = pd.DataFrame()
    good_users_5k['user_id'] = batch_df['event.payload.userid']
    good_users_5k['msg_id'] = batch_df['event.payload.msgid']
    good_users_5k['sg_event_id'] = batch_df['event.payload.sg_event_id']
    good_users_5k['subject'] = batch_df['event.payload.subject'].apply(
        lambda x: str(make_header(decode_header(x))) if pd.notnull(x) else None)
    good_users_5k['email_from'] = batch_df['event.payload.from']
    good_users_5k['email_to'] = batch_df['event.payload.email']
    good_users_5k['email_date'] = batch_df['event.payload.date']
    good_users_5k['originating_ip'] = batch_df['event.payload.originating_ip']
    good_users_5k['originating_ip_country'] = batch_df['event.payload.originating_ip'].apply(get_country)
    good_users_5k['lang'] = good_users_5k['subject'].apply(detect_lang)
    good_users_5k_processed_cols = batch_df.apply(lambda row: extract_data_from_raw_mime(row['raw_mime'], row['event.payload.msgid']), axis=1)
    good_users_5k_processed_cols = good_users_5k_processed_cols.apply(pd.Series)
    # Concatenate the extracted columns with original DataFrame
    good_users_5k = pd.concat([good_users_5k, good_users_5k_processed_cols], axis=1)

    wr.s3.to_parquet(good_users_5k, parquet_path)
    print("============================================================")
    print(f"\nParquet file stored for batch: {batch_id} at {parquet_path}\n")
    print(f"Time taken: {str(time.time() - start)}")
    print("============================================================")

In [2]:
import multiprocessing as mp
mp.cpu_count()

64

In [4]:
s3_dir = "s3://accsec-ai-prod-snitchmail/processed/not_phish/unique_user_email_data/"

# 1. List and group files (2 files = 1000 rows)
good_user_files = sorted([f for f in wr.s3.list_objects(s3_dir) if f.endswith('.parquet')])
file_groups = [good_user_files[i:i+2] for i in range(0, len(good_user_files), 2)]

In [ ]:
# 2. Create batches with metadata
batches = [
    (group, idx * 1000, idx * 1000 + 999) 
    for idx, group in enumerate(file_groups)
]

# 3. Run in parallel
with mp.Pool(processes=8) as pool:
    pool.map(process_5k_good_users, batches)

# for i in range(0, len(good_users_125k_df), PARQUET_BATCH_SIZE):
#     batch = good_users_125k_df[i: i + PARQUET_BATCH_SIZE]
#     process_5k_good_users((batch, i, min(i + PARQUET_BATCH_SIZE - 1, len(good_users_125k_df)-1)))

Uploaded to s3://accsec-ai-prod-snitchmail/processed/not_phish/html_files/6-GBHfLCSo-ewZlruPy-tQ.recvd-69cbc48b94-rsxdm-1-67DEC320-55.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/not_phish/html_files/QLj0AyOwTZKxPAF4zENWXA.recvd-69cbc48b94-k6zww-1-67D96C4A-1.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/not_phish/html_files/-18fgOA6SLiP0WbbLsWpSA.recvd-69cbc48b94-f5rdb-1-67CEDA5D-8.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/not_phish/html_files/xaRBIxjeSIylUaD2fV4XwA.recvd-69cbc48b94-ftnkf-1-67E1ABF1-8.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/not_phish/html_files/TRGwk1SqR8StYI_qtQdcog.recvd-69cbc48b94-7j96v-1-67D88D78-D.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/not_phish/html_files/9FNodyQkTDiaG7KTbA1fMw.recvd-69cbc48b94-7j96v-1-67D31F4A-15.0.html
Uploaded to s3://accsec-ai-prod-snitchmail/processed/not_phish/html_files/lyySdTimSUeYMzW6DtuT1w.recvd-69cbc48b94-6xr46-1-67D42EDB-B.0.html
Uploaded to s3://a

In [6]:
len(file_groups)

1312

In [15]:
2623*500

1311500

In [7]:
def extract_data_from_raw_mime(raw_mime, msg_id, s3_uri):
    if isinstance(raw_mime, str):
        raw_mime = raw_mime.encode('utf-8')
    msg = BytesParser(policy=policy.default).parsebytes(raw_mime)
    
    content_type = msg.get_content_type()
    text_body = ""
    html_body = ""

    def decode_part(part):
        charset = part.get_content_charset('utf-8') or 'utf-8'
        charset = charset.lower().strip().replace('3d', '').replace('=', '')
        try:
            codecs.lookup(charset)
        except LookupError:
            charset = 'utf-8'
        return part.get_payload(decode=True).decode(charset, errors='replace')

    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = part.get("Content-Disposition", "")
            if content_type == "text/plain" and "attachment" not in content_disposition:
                text_body = decode_part(part)
            elif content_type == "text/html":
                html_body = decode_part(part)
    else:
        content_type = msg.get_content_type()
        if content_type == "text/plain":
            text_body = decode_part(msg)
        elif content_type == "text/html":
            html_body = decode_part(msg)

    # If only HTML is present, convert to plain text
    if not text_body and html_body:
        soup = BeautifulSoup(html_body, 'html.parser')
        text_body = soup.get_text(separator=' ', strip=True)

    # (Optional) Save HTML to S3 if needed, as in your original code
    # ...

    return {
        "Content-Type": content_type,
        "email_text": text_body
    }


In [6]:
phish_user_emails_df = pd.read_parquet('csv_files/w_label/phish_user_emails.parquet')

In [9]:
raw_mime_sample = phish_user_emails_df[phish_user_emails_df['msg_id'] == 'UQ7u_XELTy64_FgO_jXo7Q.recvd-668fcdfd56-c6j4v-1-67C65D65-13.0'].loc[223,'mime']

In [12]:
extract_data_from_raw_mime(raw_mime_sample, '', '')

{'Content-Type': 'text/html',
 'email_text': 'Hi! This email is being sent to confirm your mailbox is active, emails are delivering successfully and it does not bounce. Please do not reply - you do not need to do anything.\r\n\r\n        Best, Smartlead team'}